In [101]:
import numpy as np
import pandas as pd
import requests
import re
# from bs4 import BeautifulSoup

In [102]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


#testing run
# url = 'https://www.countyhealthrankings.org/app/texas/2016/measure/outcomes/145/data'
# driver = webdriver.Firefox()
# driver.get(url)
# element = WebDriverWait(driver,4).until(EC.presence_of_element_located((By.XPATH, '//table[@class=\'measure-data-table sticky-enabled\']')))
# print(element.text)
# text = element.text
# driver.close()

In [103]:
state_list = ['district-of-columbia','new-mexico','north-carolina','south-carolina',
              'north-dakota','south-dakota','new-hampshire','rhode-island','new-jersey',
              'west-virginia','new-york','alabama','alaska','arizona','arkansas',
              'california','colorado','connecticut','delaware','florida','georgia',
              'idaho','illinois','indiana','iowa','kansas','kentucky','louisiana',
              'maine','maryland','massachusetts','michigan','minnesota','mississippi',
              'missouri','montana','nebraska','nevada','ohio','oklahoma','oregon',
              'pennsylvania','tennessee','texas','utah','vermont','virginia',
              'washington','wisconsin','wyoming','hawaii']


#split statelist due to cancellation by host webpage after 20 continuous scrapes
state_list1 = ['district-of-columbia','new-mexico','north-carolina','south-carolina',
              'north-dakota','south-dakota','new-hampshire','rhode-island','new-jersey',
              'west-virginia','new-york','alabama','alaska','arizona','arkansas',
              'california','colorado']

state_list2 = ['connecticut','delaware','florida','georgia','idaho','illinois','indiana',
               'iowa','kansas','kentucky','louisiana',
              'maine','maryland','massachusetts','michigan','minnesota','mississippi']

state_list3 = ['missouri','montana','nebraska','nevada','ohio','oklahoma','oregon',
              'pennsylvania','tennessee','texas','utah','vermont','virginia',
              'washington','wisconsin','wyoming','hawaii']


state_list_test = ['district-of-columbia','new-mexico','north-carolina']

In [165]:
data_list = []

In [166]:
data_list

[]

In [169]:
#loops through list of states one at a time
#opens webpage for assigned state and year in separate firefox window ,waits for javascript to populate page with data and then data scrapped


################################
#manually update 'year' variable and <2021> in url before each run
#manually change year once and listnames thrice (state_list1, state_list2, state_list3) for each year. automatic loops are getting cancelled by host
################################

for state in state_list3:
        year = '2021'  #update manually 
        url = (f'https://www.countyhealthrankings.org/app/{state}/2021/measure/outcomes/145/data') #update manually
        
        driver = webdriver.Firefox()    
        driver.get(url) #opens url in a separate Firefox window
        #wait for 4 seconds for javascript to populate data in webpage
        element = WebDriverWait(driver, 4).until(EC.presence_of_element_located((By.XPATH, '//table[@class=\'measure-data-table sticky-enabled\']')))
        d_list = [year,state,element.text]
        data_list.append(d_list)


In [157]:
#check
data_list[-1]

['2020',
 'hawaii',
 '% Frequent Mental Distress\nCounty\n  County Value\nError Margin\nHawaii 13% 12-13%\nHonolulu 9% 9-9%\nKalawao 10% 10-11%\nKauai 10% 10-10%\nMaui 11% 11-11%']

In [170]:
#create Pandas df
df_temp = pd.DataFrame(data_list, columns=['year','state','pageContent'])
df_temp.shape

(51, 3)

In [171]:
#Truncate first 62 characters from each cell
df_temp['contentLists'] = df_temp['pageContent'].apply(lambda x: x[62:].split('\n'))

In [172]:
df_temp.head()

,year,state,pageContent,contentLists
0,2021,district-of-columbia,% Frequent Mental Distress\nCounty\n County V...,[District of Columbia 13% 12-14%]
1,2021,new-mexico,% Frequent Mental Distress\nCounty\n County V...,"[Bernalillo 14% 13-15%, Catron 16% 14-17%, Cha..."
2,2021,north-carolina,% Frequent Mental Distress\nCounty\n County V...,"[Alamance 15% 14-16%, Alexander 16% 14-18%, Al..."
3,2021,south-carolina,% Frequent Mental Distress\nCounty\n County V...,"[Abbeville 17% 15-18%, Aiken 15% 14-17%, Allen..."
4,2021,north-dakota,% Frequent Mental Distress\nCounty\n County V...,"[Adams 12% 10-13%, Barnes 12% 10-13%, Benson 1..."


In [86]:
#truncating for testing purposes
df_test = df_temp.truncate(after=5)
df_test

,year,state,pageContent,contentLists
0,2016,district-of-columbia,% Frequent Mental Distress\nCounty\n County V...,[District of Columbia 10% ]
1,2016,new-mexico,% Frequent Mental Distress\nCounty\n County V...,"[Bernalillo 11% 11-11%, Catron 12% 11-12%, Cha..."
2,2016,north-carolina,% Frequent Mental Distress\nCounty\n County V...,"[Alamance 12% 12-12%, Alexander 11% 11-12%, Al..."
3,2016,south-carolina,% Frequent Mental Distress\nCounty\n County V...,"[Abbeville 13% 13-13%, Aiken 12% 12-13%, Allen..."
4,2016,north-dakota,% Frequent Mental Distress\nCounty\n County V...,"[Adams 7% 7-8%, Barnes 7% 7-8%, Benson 13% 13-..."
5,2016,south-dakota,% Frequent Mental Distress\nCounty\n County V...,"[Aurora 7% 7-8%, Beadle 8% 8-9%, Bennett 14% 1..."


In [173]:
#list to contain all data scrapped for each year
county_data_list = []

for idx,row in df_temp.iterrows():
    statename = row['state']
    year = row['year']
    content_lists = row['contentLists']
    
    for countydata in content_lists:
#         print(countydt)
        loop_list = re.split(r'\s(?=\d)', countydata)
        loop_list.insert(0,statename)
        loop_list.insert(0,year)

#         print(dataplt)
        if len(loop_list) == 4:
            loop_list[3] = loop_list[3].strip()
            loop_list.append('na')
#         print(dataplt)
        county_data_list.append(loop_list)



In [97]:
county_data_list

[['2016', 'district-of-columbia', 'District of Columbia', '10%', 'na'],
 ['2016', 'new-mexico', 'Bernalillo', '11%', '11-11%'],
 ['2016', 'new-mexico', 'Catron', '12%', '11-12%'],
 ['2016', 'new-mexico', 'Chaves', '12%', '12-12%'],
 ['2016', 'new-mexico', 'Cibola', '14%', '14-15%'],
 ['2016', 'new-mexico', 'Colfax', '12%', '11-12%'],
 ['2016', 'new-mexico', 'Curry', '11%', '11-12%'],
 ['2016', 'new-mexico', 'De Baca', '12%', '12-12%'],
 ['2016', 'new-mexico', 'Dona Ana', '13%', '13-13%'],
 ['2016', 'new-mexico', 'Eddy', '10%', '10-11%'],
 ['2016', 'new-mexico', 'Grant', '11%', '11-11%'],
 ['2016', 'new-mexico', 'Guadalupe', '11%', '11-11%'],
 ['2016', 'new-mexico', 'Harding', '11%', '11-11%'],
 ['2016', 'new-mexico', 'Hidalgo', '12%', '12-13%'],
 ['2016', 'new-mexico', 'Lea', '11%', '10-11%'],
 ['2016', 'new-mexico', 'Lincoln', '12%', '11-12%'],
 ['2016', 'new-mexico', 'Los Alamos', '8%', '8-8%'],
 ['2016', 'new-mexico', 'Luna', '14%', '13-14%'],
 ['2016', 'new-mexico', 'McKinley', '18

In [174]:
df_countyFMD21 = pd.DataFrame(county_data_list,columns=['year','state','county','fmd','error'])

In [175]:
df_countyFMD21.head()

,year,state,county,fmd,error
0,2021,district-of-columbia,District of Columbia,13%,12-14%
1,2021,new-mexico,Bernalillo,14%,13-15%
2,2021,new-mexico,Catron,16%,14-17%
3,2021,new-mexico,Chaves,15%,14-17%
4,2021,new-mexico,Cibola,18%,17-19%


In [176]:
df_countyFMD21.to_csv('countyFMD_data_2021.csv')